In [2]:
from datasets import load_dataset
ds = load_dataset("Synanthropic/reading-analog-gauge")

README.md:   0%|          | 0.00/558 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


corners.zip:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

keypoint.zip:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/42442 [00:00<?, ? examples/s]

## Unoptimized for mac m1 GPU

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt

class GaugeReader:
    def __init__(self):
        self.input_shape = (224, 224, 3)
        self.min_value = 0
        self.max_value = 10
        
    def build_model(self):
        # Modified model architecture to predict single value
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)  # Single output for gauge reading
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        # Load dataset from HuggingFace
        print("Loading dataset...")
        dataset = load_dataset("Synanthropic/reading-analog-gauge")
        train_ds = dataset['train']
        
        def preprocess_single_example(example):
            # Convert image to numpy array
            image = np.array(example['image'])
            
            # Resize image
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            
            # Normalize image
            image = image.astype(np.float32) / 255.0
            
            # Get label
            label = np.array(example['label'], dtype=np.float32)
            
            return {
                'image': image,
                'label': label
            }
        
        print("Processing examples...")
        # Process all examples
        processed_data = [preprocess_single_example(example) for example in train_ds]
        
        # Separate images and labels
        images = np.array([example['image'] for example in processed_data])
        labels = np.array([example['label'] for example in processed_data])
        
        print(f"Dataset shape - Images: {images.shape}, Labels: {labels.shape}")
        
        # Create TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices((images, labels))
        
        # Batch and shuffle
        dataset = dataset.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
        
        return dataset

    def train_model(self, epochs=10):
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds = self.prepare_dataset()
        print("Dataset prepared successfully!")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='loss',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    factor=0.5,
                    patience=2
                )
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)
    
    def predict_value(self, model, image_path):
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
        image = image.astype(np.float32) / 255.0
        
        # Add batch dimension
        image = np.expand_dims(image, 0)
        
        # Predict
        prediction = model.predict(image)
        return float(prediction[0][0])

    def visualize_prediction(self, image_path, prediction):
        # Load and resize image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
        
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f'Predicted Value: {prediction:.2f}')
        plt.axis('off')
        plt.show()

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    # Train model
    print("Training model...")
    model, history = gauge_reader.train_model(epochs=10)
    
    # Convert to TFLite
    print("Converting to TFLite...")
    tflite_model = gauge_reader.convert_to_tflite(model)
    
    # Save model
    print("Saving model...")
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='MAE')
    plt.title('Model MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

Training model...
Preparing dataset...
Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.


Processing examples...


In [12]:
# Check dataset structure
dataset = load_dataset("Synanthropic/reading-analog-gauge")
train_ds = dataset['train']
example = train_ds[0]
print("Dataset keys:", example.keys())

Repo card metadata block was not found. Setting CardData to empty.


Dataset keys: dict_keys(['image', 'label'])


## Optimized for mac m1 GPU

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt
import gc  # Garbage collector

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 16
        
    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            split='train'
        )
        
        print(f"Total examples in dataset: {len(dataset)}")
        
        def preprocess_example(example):
            # Load and resize image
            image = np.array(example['image'])
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            image = image.astype(np.float32) / 255.0
            label = float(example['label'])
            return image, label

        # Create tf.data.Dataset directly from the dataset
        def generator():
            for example in dataset:
                image, label = preprocess_example(example)
                yield image, label

        # Create dataset with the correct shapes and types
        tf_dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )

        # Shuffle, batch, and prefetch
        tf_dataset = tf_dataset.shuffle(1000)
        tf_dataset = tf_dataset.batch(self.batch_size)
        tf_dataset = tf_dataset.prefetch(tf.data.AUTOTUNE)

        # Calculate steps per epoch
        steps_per_epoch = len(dataset) // self.batch_size

        return tf_dataset, steps_per_epoch

    def train_model(self, epochs=10):
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds, steps_per_epoch = self.prepare_dataset()
        print(f"Dataset prepared successfully! Steps per epoch: {steps_per_epoch}")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='loss',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    factor=0.5,
                    patience=2
                ),
                # Memory cleanup callback
                tf.keras.callbacks.LambdaCallback(
                    on_epoch_end=lambda epoch, logs: gc.collect()
                )
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        print("Converting to TFLite format...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    try:
        # Train model
        print("Training model...")
        model, history = gauge_reader.train_model(epochs=10)
        
        # Convert to TFLite
        print("Converting to TFLite...")
        tflite_model = gauge_reader.convert_to_tflite(model)
        
        # Save model
        print("Saving model...")
        gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
        
        # Plot training history
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['mae'], label='MAE')
        plt.title('Model MAE')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        # Clean up memory in case of error
        gc.collect()

if __name__ == "__main__":
    main()

Training model...
No GPU found or GPU memory growth setting failed
Preparing dataset...
Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.


An error occurred: The dataset length is unknown.


## optimized for RTX 3060 gpu

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)  # Reduced input size
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 32  # Smaller batch size to reduce memory

    def build_model(self):
        # Lighter model architecture with fewer parameters
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            
            # First conv block - reduced filters
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            # Second conv block
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            # Third conv block
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            # Fourth conv block
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            
            # Reduced dense layers
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1)
        ])
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
        model.compile(
            optimizer=optimizer,
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        # Load dataset in streaming mode
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            streaming=True
        )
        train_ds = dataset['train']
        
        def preprocess_example(example):
            image = np.array(example['image'])
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            image = (image.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
            return image, example['label']
        
        # Create streaming datasets with generators
        def generate_examples(split):
            for example in split:
                yield preprocess_example(example)
        
        # Calculate approximate sizes
        total_size = 10000  # Approximate dataset size
        val_size = int(0.1 * total_size)
        
        # Create datasets using from_generator
        train_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.take(total_size - val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        val_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.skip(total_size - val_size).take(val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        # Optimize for performance while maintaining memory efficiency
        train_dataset = (train_dataset
            .shuffle(1000)  # Reduced buffer size
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        val_dataset = (val_dataset
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        return train_dataset, val_dataset

    def train_model(self, epochs=20):
        # Configure memory growth
        for device in tf.config.list_physical_devices('GPU'):
            tf.config.experimental.set_memory_growth(device, True)
        
        model = self.build_model()
        train_ds, val_ds = self.prepare_dataset()
        
        # Add TensorBoard callback for memory monitoring
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir='./logs',
            profile_batch='500,520'  # Profile a few batches
        )
        
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-6
                ),
                tensorboard_callback
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]  # Use FP16 quantization
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            f.write(tflite_model)  # Save directly without pickle

def main():
    gauge_reader = GaugeReader()
    model, history = gauge_reader.train_model(epochs=20)
    tflite_model = gauge_reader.convert_to_tflite(model)
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot with reduced memory usage
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train')
    plt.plot(history.history['val_mae'], label='Val')
    plt.title('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

if __name__ == "__main__":
    main()

/home/faizal/work/gnprc-domi-app/tf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1735302132.971772    6181 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9812 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:2b:00.0, compute capability: 8.6


Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.
2024-12-27 17:52:28.489984: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2024-12-27 17:52:28.490012: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2024-12-27 17:52:28.490036: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1006] Profiler found 1 GPUs
2024-12-27 17:52:28.518519: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2024-12-27 17:52:28.518593: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1213] CUPTI activity buffer flushed


Epoch 1/20


2024-12-27 17:52:40.442151: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 5 of 1000
2024-12-27 17:52:50.781367: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 12 of 1000
2024-12-27 17:53:10.919037: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 26 of 1000
2024-12-27 17:53:21.097728: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 33 of 1000
2024-12-27 17:53:40.070189: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 46 of 1000
2024-12-27 17:53:50.271973: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 53 o

      1/Unknown 1539s 1539s/step - loss: 0.0012 - mae: 0.0280

I0000 00:00:1735303687.475291    7506 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


     63/Unknown 4542s 48s/step - loss: 0.0024 - mae: 0.0211

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-12-27 17:51:51.975267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735302112.075544    6181 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735302112.103413    6181 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 17:51:52.359926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


In [ ]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 64  # Increased but still conservative for 12GB VRAM
        
        # Enable mixed precision for better memory efficiency
        mixed_precision.set_global_policy('mixed_float16')

    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1)
        ])
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
        model.compile(
            optimizer=optimizer,
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            streaming=True
        )
        train_ds = dataset['train']
        
        @tf.function
        def preprocess_image(image):
            image = tf.image.resize(image, [self.input_shape[0], self.input_shape[1]])
            image = tf.cast(image, tf.float32) / 127.5 - 1
            return image

        def generate_examples(split):
            for example in split:
                # Convert PIL image to numpy array
                image = np.array(example['image'])
                # Convert to tensor and preprocess
                image = preprocess_image(image)
                yield image, example['label']

        # Calculate sizes
        total_size = 10000
        val_size = int(0.1 * total_size)
        
        # Create datasets
        train_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.take(total_size - val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        val_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.skip(total_size - val_size).take(val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        # Optimize the input pipeline
        train_dataset = (train_dataset
            .cache()  # Cache after preprocessing
            .shuffle(1000)
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        val_dataset = (val_dataset
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        return train_dataset, val_dataset

    def train_model(self, epochs=20):
        # Configure GPU memory growth
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        
        model = self.build_model()
        train_ds, val_ds = self.prepare_dataset()
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir='./logs',
            profile_batch='100,120'
        )
        
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-6
                ),
                tensorboard_callback
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            f.write(tflite_model)

def main():
    gauge_reader = GaugeReader()
    model, history = gauge_reader.train_model(epochs=20)
    tflite_model = gauge_reader.convert_to_tflite(model)
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot with reduced memory usage
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train')
    plt.plot(history.history['val_mae'], label='Val')
    plt.title('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

if __name__ == "__main__":
    main()

Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.


Epoch 1/20


2024-12-27 19:12:05.432159: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2024-12-27 19:12:05.432183: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2024-12-27 19:12:05.437055: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2024-12-27 19:12:05.440574: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1213] CUPTI activity buffer flushed
2024-12-27 19:12:17.525228: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:19: Filling up shuffle buffer (this may take a while): 4 of 1000
2024-12-27 19:12:27.841763: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:19: Filling up shuffle buffer (this may take a while): 11 of 1000
2024-12-27 19:12:48.472231: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:19: Filling up shuffle buffer (this may take a while): 24 of 1000
2024-12-2

     11/Unknown 2565s 97s/step - loss: 2.6546 - mae: 1.1359

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 01e050d6-887c-4282-a4c1-2fb05ea4c2a4)')' thrown while requesting GET https://huggingface.co/datasets/Synanthropic/reading-analog-gauge/resolve/f9b31b44c9a693eddcc9c93a247ad30c3005ea07/corners.zip
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/Synanthropic/reading-analog-gauge/resolve/f9b31b44c9a693eddcc9c93a247ad30c3005ea07/corners.zip (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7788333d4200>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 1a075cb6-9d14-46ac-9005-ab8914b3560c)')' thrown while requesting GET https://huggingface.co/datasets/Synanthropic/reading-analog-gauge/resolve/f9b31b44c9a693eddcc9c93a247ad30c3005ea07/corners.zip
Retrying in 2s [Retry 2/5].
'(MaxRetryError(

     31/Unknown 4583s 100s/step - loss: 1.6418 - mae: 0.8470

In [3]:
from mlcroissant import Dataset
ds = Dataset(jsonld="https://huggingface.co/api/datasets/Synanthropic/reading-analog-gauge/croissant")
records = ds.records("default")

# Print available record names/tables
print("Available tables:", ds.metadata)

# Print dataset metadata
print("\nDataset metadata:")
print(ds.metadata)

# Try to access records properly
try:
    # Different ways to try accessing the records
    for record in records:
        print("\nFirst record structure:", record)
        break
except Exception as e:
    print(f"\nError accessing records directly: {e}")

# Try to get schema information
print("\nSchema information:")
print(records.schema if hasattr(records, 'schema') else "No schema available")

# Print any available fields or columns
print("\nAvailable fields:")
for field in records.fields if hasattr(records, 'fields') else []:
    print(f"- {field}")

  -  [Metadata(reading-analog-gauge)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(reading-analog-gauge)] Property "https://schema.org/datePublished" is recommended, but does not exist.
  -  [Metadata(reading-analog-gauge)] Property "https://schema.org/license" is recommended, but does not exist.
  -  [Metadata(reading-analog-gauge)] Property "https://schema.org/version" is recommended, but does not exist.


Available tables: None

Dataset metadata:
Metadata(uuid="_:N47895c671c5e49338a32181b5b015fb5")

Error accessing records directly: An error occured during the sequential generation of the dataset, more specifically during the operation Download(repo)

Schema information:
No schema available

Available fields:


## Latest model on the roboflow dataset

In [ ]:
import tensorflow as tf
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from object_detection.builders import model_builder
from object_detection.utils import config_util
import numpy as np
import cv2
import math

class GaugeReaderTF:
    def __init__(self, label_map_path):
        self.label_map = label_map_util.load_labelmap(label_map_path)
        self.categories = label_map_util.convert_label_map_to_categories(
            self.label_map, max_num_classes=3, use_display_name=True)
        self.category_index = label_map_util.create_category_index(self.categories)
        
    def parse_tfrecord_fn(self, example_proto):
        """Parse TFRecord file format"""
        feature_description = {
            'image/height': tf.io.FixedLenFeature([], tf.int64),
            'image/width': tf.io.FixedLenFeature([], tf.int64),
            'image/encoded': tf.io.FixedLenFeature([], tf.string),
            'image/format': tf.io.FixedLenFeature([], tf.string),
            'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
            'image/object/class/label': tf.io.VarLenFeature(tf.int64)
        }
        
        example = tf.io.parse_single_example(example_proto, feature_description)
        
        # Get the image
        image = tf.io.decode_raw(example['image/encoded'], tf.uint8)
        height = tf.cast(example['image/height'], tf.int32)
        width = tf.cast(example['image/width'], tf.int32)
        image = tf.reshape(image, [height, width, 3])
        
        # Get the labels
        labels = tf.cast(example['image/object/class/label'].values, tf.int32)
        
        # Get the bounding boxes
        xmins = example['image/object/bbox/xmin'].values
        xmaxs = example['image/object/bbox/xmax'].values
        ymins = example['image/object/bbox/ymin'].values
        ymaxs = example['image/object/bbox/ymax'].values
        
        return {
            'image': image,
            'labels': labels,
            'bboxes': tf.stack([xmins, ymins, xmaxs, ymaxs], axis=1)
        }

    def create_dataset(self, tfrecord_path, batch_size=8):
        """Create a dataset from TFRecord file"""
        dataset = tf.data.TFRecordDataset(tfrecord_path)
        dataset = dataset.map(self.parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(buffer_size=1000)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset

    def build_model(self):
        """Build and return a pre-trained object detection model"""
        # Using SSD ResNet50 V1 FPN from TensorFlow Model Garden
        pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
        detection_model = model_builder.build(
            model_config=config_util.get_configs_from_pipeline_file(pipeline_config)['model'],
            is_training=True
        )
        return detection_model

    def calculate_angle(self, center, needle_tip):
        """Calculate angle between vertical line and needle"""
        dx = needle_tip[0] - center[0]
        dy = needle_tip[1] - center[1]
        angle = math.degrees(math.atan2(dy, dx))
        if angle < 0:
            angle += 360
        return angle

    def get_reading_from_angle(self, angle, min_angle=45, max_angle=315, min_value=0, max_value=100):
        """Convert angle to gauge reading"""
        # Normalize the angle to the gauge's range
        if angle > max_angle:
            angle -= 360
            
        # Calculate reading using linear interpolation
        angle_range = max_angle - min_angle
        value_range = max_value - min_value
        
        reading = ((angle - min_angle) / angle_range) * value_range + min_value
        return round(reading, 1)

    def process_predictions(self, predictions, image):
        """Process model predictions to get gauge reading"""
        height, width = image.shape[:2]
        boxes = predictions['detection_boxes'][0].numpy()
        scores = predictions['detection_scores'][0].numpy()
        classes = predictions['detection_classes'][0].numpy().astype(int)
        
        # Initialize variables to store detection results
        center = None
        needle_tip = None
        gauge_box = None
        
        # Process each detection with score > 0.5
        for box, score, class_id in zip(boxes, scores, classes):
            if score > 0.5:
                ymin, xmin, ymax, xmax = box
                # Convert normalized coordinates to pixel coordinates
                xmin = int(xmin * width)
                xmax = int(xmax * width)
                ymin = int(ymin * height)
                ymax = int(ymax * height)
                
                if class_id == 1:  # Center
                    center = (int((xmin + xmax) / 2), int((ymin + ymax) / 2))
                elif class_id == 2:  # Gauge
                    gauge_box = (xmin, ymin, xmax, ymax)
                elif class_id == 3:  # Needle
                    needle_tip = (int((xmin + xmax) / 2), int((ymin + ymax) / 2))
        
        if center and needle_tip:
            angle = self.calculate_angle(center, needle_tip)
            reading = self.get_reading_from_angle(angle)
            
            return {
                'center': center,
                'needle_tip': needle_tip,
                'gauge_box': gauge_box,
                'angle': angle,
                'reading': reading
            }
        return None

    def visualize_results(self, image, results):
        """Visualize detection results on the image"""
        output = image.copy()
        
        if results['gauge_box']:
            x1, y1, x2, y2 = results['gauge_box']
            cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        if results['center']:
            cv2.circle(output, results['center'], 5, (0, 0, 255), -1)
            
        if results['needle_tip']:
            cv2.circle(output, results['needle_tip'], 5, (255, 0, 0), -1)
            cv2.line(output, results['center'], results['needle_tip'], (255, 0, 0), 2)
            
        # Add text with reading
        cv2.putText(output, f"Reading: {results['reading']}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        return output

In [ ]:
# Initialize the gauge reader
gauge_reader = GaugeReaderTF('path/to/Gauge_label_map.pbtxt')

# Create datasets
train_dataset = gauge_reader.create_dataset('../datasets/Analog Gauge Meter.v13-v06-crop.tfrecord/train/Guage.tfrecord')
valid_dataset = gauge_reader.create_dataset('../datasets/Analog Gauge Meter.v13-v06-crop.tfrecord/valid/Gauge.tfrecord')

# Build the model
model = gauge_reader.build_model()

# Train the model (you'll need to implement the training loop based on your specific requirements)
# ...

# Make predictions on a single image
image = cv2.imread('.jpg')
predictions = model(tf.convert_to_tensor([image]))
results = gauge_reader.process_predictions(predictions, image)

# Visualize results
if results:
    output_image = gauge_reader.visualize_results(image, results)
    cv2.imwrite('result.jpg', output_image)
    print(f"Gauge Reading: {results['reading']}")